akv17

## Коротко о концепте

В целях унификации выдачи каждого *корпуса / словаря / чего-то еще* удобно ввести объекты типа __`Result`__ и __`Target`__.<br>
Тип __`Result`__ содержит все результаты одного запроса.<br>
Тип __`Target`__ является оболочкой для одного целевого вхождения (найденного слова, соответствующего запросу).<br>
Выдача произвольного *корпуса / словаря / чего-то еще* представлена в виде одного объекта типа __`Result`__, содержащего $N$ объектов типа __`Target`__, где $N$ = `numResults`.<br>

Помимо этого был переписан тип __`Query`__ , в результате чего упразднены типы __`Manager`__, __`Session`__ и __`Downloader`__ (*__Downloader__ по желанию можно оставить*)

## Ключевые изменения

- __PageParser__ произвольного *корпуса / словаря / чего-то еще* должен иметь метод `.extract()`, с помощью которого объекты типа __`Target`__ для каждого найденного целевого вхождения are streamed via `yield` напрямую в __`Query`__`.search()`. Таким образом, `.extract()` является генератором, передающим объекты типа __`Target`__. Более никаких ограничений на архитектуру __PageParser__ нет.
- тип __Query__ инициализируется только одним параметром `language`.
- все возможные параметры всех реализованных *корпусов / словарей / чего-то еще* хранятся в __`params_container.Container`__ 

## Прототип создания объекта типа Target

Требуется предоставить 4 параметра:
- строку всего предложения / документа с текущим целевым вхождением как параметр `text`
- индексы начала и конца целевого вхождения в `.text` на символьном уровне как параметр `idxs`<br>
      пример: пусть `query = 'два'`, `.text = 'раз, два и три!'`, тогда `.idxs = (5, 8)`, т.к. `.text[5:8] = 'два'`
- информация о предложении / документе как параметр `meta` (давайте пустую строку, если нету информации)
- тэги целевого вхождения как параметр `tags` (по умолчанию как тип `dict`; пустой список, если нет)

__ВАЖНО__: ожидается один объект типа __Target__ на одно целевое вхождение. Как следствие, если у вас более одного целевого вхождения в одном предложении / документе, разбейте их на несколько объектов типа __Target__.

In [ ]:
class Target:
    def __init__(self, text, idxs, meta, tags):
        """
        text: str: full sentence / document
        idxs: tuple (l, r): target idxs in self.text -> self.text[l:r]
        meta: str: sentence / document info
        tags: dict?: target tags
        """
        self.text = text
        self.idxs = idxs
        self.meta = meta
        self.tags = tags
    
    # ...

## Прототип PageParser

In [ ]:
class PageParser(Container):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # ...
    
    def любой_ваш_метод_для_получения_результатов(self):
        pass
    
    # ...
    
    def любой_ваш_метод_для_получения_результатов_10(self):
        pass
    
    def extract(self):
        """
        --- ГЕНЕРАТОР ПО НАЙДЕННЫМ ЦЕЛЕВЫМ ВХОЖДЕНИЯМ
            КАК ТИПАМ Target.
            сюда обращается Query.search() ---
        """
        # ...
        
        # для каждого найденного целевого
        # вхождения отдадим в Query.search()
        # объект типа Target,
        # описывающий данное найденное целевое вхождение
        for text, idxs, meta, tags in found:
            yield Target(text, idxs, meta, tags)

## Как собрать тестовый билд у себя:
- 0: Положить в одно место __query.py__, __params_container.py__, __target.py__, и __result.py__ 
- 1: Положить в это же место .py скрипт вашего *корпуса / словаря / чего-то еще*
- 2: Импортнуть тип __Target__ в .py вашего *корпуса / словаря / чего-то еще*: `from target import Target`
- 3: Импортнуть ваш .py *корпуса / словаря / чего-то еще* в __query.py__: `import my_corpus`
- 4: Добавить alias вашего *корпуса / словаря / чего-то еще* в `functions` как ключ (этот ключ передается в `Query` как `language`): `functions = {'my_corp': my_corpus}`
- 5: В любом редакторе: `import query / from query import Query` --> готово

## Краткое руководство использования
- инициализировать объект типа __Query__ для работы с определенным *языком / словарем / чем-то еще*
- сделать запрос / мультизапрос через `Query.search()`
- получить выдачу как список типов __Result__ для запроса / каждого из запросов (мультизапрос)
- получить список типов __Target__ как `Result.results` 

### Пример

In [1]:
from query import Query

In [2]:
q = Query(language='rus')

In [3]:
r = q.search('фонема', numResults=10)

Query "фонема": 100%|████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.42docs/s]


In [4]:
r

[Result(фонема, 10)]

In [5]:
r[0].results

[Target(фоне́ма, А. А. Зализняк. Лингвистика по А. Т. Фоменко // «Вопросы языкознания», 2000),
 Target(фонема, В. А. Успенский. Математическое и гуманитарное: преодоление барьера (2007-2011)),
 Target(фонема, Андрей Зализняк. О профессиональной и любительской лингвистике // «Наука и жизнь», 2009),
 Target(фонема, Андрей Зализняк. О профессиональной и любительской лингвистике // «Наука и жизнь», 2009),
 Target(Фонема, А. В. Суперанская. Вновь о букве Ё // «Наука и жизнь», 2008),
 Target(фонема, Михаил Гиголашвили. Красный озноб Тингитаны: Записки о Марокко (2006) // «Нева», 2008),
 Target(фонема, Л. А. Гоготишвили. «Эйдетический язык» (реконструкция и интерпретация радикальной феноменологической новации А. Ф. Лосева) (2006)),
 Target(фонема, Андрей Геласимов. Рахиль (2004) // «Октябрь», 2003),
 Target(фонема, Михаил Эпштейн. Поэтика близости // «Звезда», 2003),
 Target(фонема, [Staples] (2003) // «Рекламный мир», 2003.04.28)]

In [6]:
for i, _r in enumerate(r[0].results): print('%s: %s' % (i + 1, _r.text))

1:  Для на́шего изложе́ния доста́точно счита́ть, что лингвисти́ческий те́рмин "фоне́ма" есть про́сто не́которое уточне́ние поня́тия "звук языка́". 
2:  В качестве исходных отношений ―  , в каковом отношении могли находиться две цепочки звуков языка и  , в каковом отношении могли находиться звук языка и фонема. 
3:   *Что касается термина «фонема», то здесь нам достаточно считать, что это просто некоторое уточнение понятия «звук языка». 
4:   Принцип состоит в том, что внешняя форма слов языка меняется не индивидуальным образом для каждого слова, а в силу процессов ― так называемых фонетических изменений (иначе ― фонетических переходов), охватывающих в данном языке в данную эпоху ВСЕ без исключения слова, где имеется определённая фонема (или сочетание фонем). 
5:  Фонема э под ударением после мягких согласных и шипящих превратилась в фонему о: пошёл, лжёшь, пшённый, пчёл, жён, козёл, нёс. 
6:  Реальная, звучная фонема, основательная во всех смыслах. 
7:  То, как структурирована фонема в

In [8]:
# kwic
for i, _r in enumerate(r[0].results): print('%s: %s \t %s \t %s' % (i + 1, *_r.kwic(5, 5)))

1: доста́точно счита́ть, что лингвисти́ческий те́рмин" 	 фонема 	 "есть про́сто не́которое уточне́ние поня́тия
2: могли находиться звук языка и 	 фонема 	 .
3: *Что касается термина« 	 фонема 	 »,то здесь нам достаточно считать,
4: исключения слова, где имеется определённая 	 фонема 	 (или сочетание фонем).
5:  	 Фонема 	 э под ударением после мягких
6: Реальная, звучная 	 фонема 	 ,основательная во всех смыслах.
7: То, как структурирована 	 фонема 	 в качестве «пучка дифференциалов», или
8: Первая гласная 	 фонема 	 произносится как звук «е», а
9: структурных единиц языка: «лексема, морфема, 	 фонема 	 »и пр.
10: ням», делаю вывод, что данная 	 фонема 	 не способствует запоминанию торговой марки.


In [3]:
r2 = q.search(['велярный', 'йот'], numResults=5, tag=True) 

Query "велярный":   0%|                                                                        | 0/5 [00:00<?, ?docs/s]
C:\Users\PC\lingcorpora\build\lingcorpora.py:63: UserWarning: Nothing found for query "велярный".
Unsuccessful queries are available via Query.unsuccessful
  warnings.warn(self.__warn % q)
Query "йот": 100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.39docs/s]


In [4]:
r2

[Result(велярный, 0), Result(йот, 5)]

In [8]:
q.unsuccessful

['велярный']

In [5]:
r2[1].results[0].tags

{'flags': ['adash',
  'amark',
  'casered',
  'genderred',
  'norm',
  'numred',
  'posred'],
 'gramm': ['S', 'inan', 'f', 'pl', 'gen', 'disamb'],
 'lex': ['йота'],
 'sem': ['r:concr', 't:letter']}